In [2]:
%pip install -q accelerate peft bitsandbytes transformers trl tensorboard huggingface_hub[cli] xformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.13.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.14.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%%capture
%pip install sentencepiece

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import os
import torch
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    AutoTokenizer,
    LlamaTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer
import transformers

2023-10-10 16:56:56.736661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 16:56:57.016233: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-10 16:56:57.915274: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-10 16:56:57.915368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /home/user/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Loading binary /home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/user/anaconda3/lib/libcudart.so.11.0'), PosixPath('/home/user/anaconda3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
/home/user/anacond

In [ ]:
!pip install 

In [2]:
SEED = 42
ITERS = 1000
MIN_TOKENS = 180
MAX_TOKENS = 200
MODEL_NAME = "meta-llama/Llama-2-7b-hf"
DATASET_PATH = f"bias_llm/data/llama2-7b-200-economy-infra"
# TOPICS = [
#   "Indian climate",
#   "Indian defense",
#   "Indian economy",
#   "Indian infrastructure",
# ]
TOPICS = [
  "Indian economy",  
  "Indian infrastructure",
]

In [3]:
def get_transformer() -> tuple:
  tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

  # Load the base model
  model = LlamaForCausalLM.from_pretrained(
      MODEL_NAME,
      device_map={"": 0},
  )

  return tokenizer, model

In [4]:
def get_encoding(tokenizer, prompt: str) -> dict:
  return tokenizer(prompt, return_tensors='pt')

def get_text(tokenizer, embedding: dict) -> str:
  return tokenizer.batch_decode(
    embedding,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
  )[0]

In [5]:
def generate_text(tokenizer, model, prompt) -> None:
  create_directory(prompt)
  create_csv_file(prompt)

  input_embedding = get_encoding(tokenizer, prompt)

  for i in range(ITERS):
    MAX_LEN = random.randint(MIN_TOKENS, MAX_TOKENS)
    print(f'Generating text for "{prompt}"... ({i + 1}/{ITERS})')

    output_embedding = model.generate(
      input_embedding.input_ids.to('cuda'),
      max_new_tokens=MAX_LEN,
      no_repeat_ngram_size = 2,
      do_sample=True,
      top_k=40,
      top_p=0.9,
      temperature=0.6, # randomly chosen
      eos_token_id=tokenizer.eos_token_id
    )

    generated_text = get_text(tokenizer, output_embedding)
    append_to_csv_file(prompt, generated_text, i, MAX_LEN)

def create_directory(prompt: str) -> None:
  if not os.path.exists(f'{DATASET_PATH}/{prompt}'):
    os.makedirs(f'{DATASET_PATH}/{prompt}')

  print(f'Created directory for "{prompt}" at {DATASET_PATH}/{prompt}...')

def create_csv_file(prompt: str) -> None:
  fields = ['Prompt', 'Generated Text', 'i', 'MAX_LEN']

  with open(f'{DATASET_PATH}/{prompt}/data.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields, lineterminator='\n')
    writer.writeheader()

  print(f'Created CSV file for "{prompt}" at {DATASET_PATH}/{prompt}/data.csv...')

def append_to_csv_file(prompt: str, generated_text: str, i: int, MAX_LEN: int) -> None:
  with open(f'{DATASET_PATH}/{prompt}/data.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow([prompt, generated_text, i, MAX_LEN])

def main() -> None:
  threads = []
  tokenizer, model = get_transformer()

  for topic in TOPICS:
    thread = Thread(target=generate_text, args=(tokenizer, model, topic,))
    thread.start()
    threads.append(thread)

  for thread in threads:
    thread.join()

In [6]:
from threading import Thread
from time import perf_counter

import os
import csv
import random
import torch

torch.manual_seed(SEED)

start_time = perf_counter()
print("Starting...")
main()
print("Done!")
end_time = perf_counter()

print(f"Time Taken: {end_time - start_time:0.4f} second(s) elapsed")

Starting...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Created directory for "Indian economy" at bias_llm/data/llama2-7b-200-economy-infra/Indian economy...
Created directory for "Indian infrastructure" at bias_llm/data/llama2-7b-200-economy-infra/Indian infrastructure...
Created CSV file for "Indian infrastructure" at bias_llm/data/llama2-7b-200-economy-infra/Indian infrastructure/data.csv...
Generating text for "Indian infrastructure"... (1/1000)
Created CSV file for "Indian economy" at bias_llm/data/llama2-7b-200-economy-infra/Indian economy/data.csv...
Generating text for "Indian economy"... (1/1000)
Generating text for "Indian infrastructure"... (2/1000)
Generating text for "Indian economy"... (2/1000)
Generating text for "Indian infrastructure"... (3/1000)
Generating text for "Indian economy"... (3/1000)
Generating text for "Indian infrastructure"... (4/1000)
Generating text for "Indian economy"... (4/1000)
Generating text for "Indian infrastructure"... (5/1000)
Generating text for "Indian economy"... (5/1000)
Generating text for "In

In [14]:
ITERS = 2000

from threading import Thread
from time import perf_counter

import os
import csv
import random
import torch

torch.manual_seed(SEED)

start_time = perf_counter()
print("Starting...")
main()
print("Done!")
end_time = perf_counter()

print(f"Time Taken: {end_time - start_time:0.4f} second(s) elapsed")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 47.54 GiB total capacity; 40.21 GiB already allocated; 81.31 MiB free; 40.22 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
import torch

In [18]:
[torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [8]:
import pandas as pd

In [9]:
dfs = {}

for topic in TOPICS:
  df = pd.read_csv(f"{DATASET_PATH}/{topic}/data.csv")
  dfs[topic] = df

In [10]:
dfs['Indian economy']

,Prompt,Generated Text,i,MAX_LEN
0,Indian economy,Indian economy likely to grow 8-8.5% on strong...,0,47
1,Indian economy,Indian economy is in doldrums due to weak GDP ...,1,57
2,Indian economy,"Indian economy will soon bounce back, says Jai...",2,46
3,Indian economy,Indian economy slows down as demand weakens\nS...,3,58
4,Indian economy,Indian economy's potential to create more than...,4,54
...,...,...,...,...
1995,Indian economy,Indian economy grows 9.5% in Q2\nNew Delhi (AN...,1995,57
1996,Indian economy,Indian economy likely to get ‘strong fillip’ a...,1996,52
1997,Indian economy,Indian economy to grow at fastest rate among e...,1997,54
1998,Indian economy,Indian economy likely to grow by 5-5.5 per cen...,1998,51


In [11]:
dfs['Indian climate']

,Prompt,Generated Text,i,MAX_LEN
0,Indian climate,Indian climate is tropical and monsoon with a ...,0,49
1,Indian climate,Indian climate - A great place for wine? - Par...,1,57
2,Indian climate,Indian climate experts to brief Bangladeshi cl...,2,50
3,Indian climate,Indian climate has four main seasons i.e. summ...,3,52
4,Indian climate,Indian climate\nThe climate in India is mainly...,4,55
...,...,...,...,...
1995,Indian climate,"Indian climate, economy to suffer in absence o...",1995,48
1996,Indian climate,Indian climate.\n(Indian climate is characteri...,1996,50
1997,Indian climate,Indian climate change\nI. K. MANDALA\nTHE firs...,1997,46
1998,Indian climate,Indian climate scientist: Rice has adapted to ...,1998,49


In [12]:
dfs['Indian defense']

,Prompt,Generated Text,i,MAX_LEN
0,Indian defense,Indian defense contractor to display cutting-e...,0,47
1,Indian defense,Indian defense secretary arrives for Afghan se...,1,56
2,Indian defense,"Indian defense deals, India’s foreign policy, ...",2,51
3,Indian defense,Indian defense contractor Hindustan Aeronautic...,3,60
4,Indian defense,"Indian defense minister meets China’s Jinping,...",4,60
...,...,...,...,...
1995,Indian defense,Indian defense is trying to figure out ways to...,1995,46
1996,Indian defense,"Indian defense, finance ministers urge BRICS t...",1996,49
1997,Indian defense,Indian defense has been in doldrums after defe...,1997,55
1998,Indian defense,Indian defense personnel receive French-made L...,1998,59


In [13]:
dfs['Indian infrastructure']

,Prompt,Generated Text,i,MAX_LEN
0,Indian infrastructure,Indian infrastructure: A tale of two cities\nI...,0,52
1,Indian infrastructure,Indian infrastructure to see a sea change duri...,1,47
2,Indian infrastructure,Indian infrastructure has been improving but w...,2,58
3,Indian infrastructure,Indian infrastructure investment funds and alt...,3,58
4,Indian infrastructure,Indian infrastructure projects are likely to b...,4,58
...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure growth could be a good i...,1995,53
1996,Indian infrastructure,Indian infrastructure sector was at the boom w...,1996,50
1997,Indian infrastructure,Indian infrastructure funds attractive: KKR\nT...,1997,47
1998,Indian infrastructure,Indian infrastructure growth is slow and unsus...,1998,48


In [14]:
df_eco = dfs['Indian economy']
texts = df_eco['Generated Text']

In [15]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>
[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 104] Connection reset by peer>


False

In [17]:
for text in texts[8:]:
  print('text:', text)
  sentences = sent_tokenize(text)
  for sentence in sentences:
    print('------')
    print('sentence:', sentence)
    words = word_tokenize(sentence)
    print('pos tags:', nltk.pos_tag(words))
    words = [word.lower() for word in words if word not in stopwords.words('english') and word.isalpha()]
    print('processed:', words)
  break

text: Indian economy not in recession and has room for recovery: RBI Governor
Sugata Ghosh / TNN / Jun 10, 2021,
Mumbai: Reserve Bank of India governor Shaktik
------
sentence: Indian economy not in recession and has room for recovery: RBI Governor
Sugata Ghosh / TNN / Jun 10, 2021,
Mumbai: Reserve Bank of India governor Shaktik
pos tags: [('Indian', 'JJ'), ('economy', 'NN'), ('not', 'RB'), ('in', 'IN'), ('recession', 'NN'), ('and', 'CC'), ('has', 'VBZ'), ('room', 'NN'), ('for', 'IN'), ('recovery', 'NN'), (':', ':'), ('RBI', 'NNP'), ('Governor', 'NNP'), ('Sugata', 'NNP'), ('Ghosh', 'NNP'), ('/', 'NNP'), ('TNN', 'NNP'), ('/', 'NNP'), ('Jun', 'NNP'), ('10', 'CD'), (',', ','), ('2021', 'CD'), (',', ','), ('Mumbai', 'NNP'), (':', ':'), ('Reserve', 'NNP'), ('Bank', 'NNP'), ('of', 'IN'), ('India', 'NNP'), ('governor', 'NN'), ('Shaktik', 'NNP')]
processed: ['indian', 'economy', 'recession', 'room', 'recovery', 'rbi', 'governor', 'sugata', 'ghosh', 'tnn', 'jun', 'mumbai', 'reserve', 'bank', 'i